In [1]:
#hide
#default_exp actions
from nbdev.showdoc import show_doc

# Actions

> Functions and scripts that act on a KiCad project.

* toc: true

In [2]:
#export
import os
import sys
import pkg_resources
import subprocess

import jinja2
from fastcore.script import *
import pandas as pd

from kicad_helpers import *
from kicad_helpers.utilities import _set_root

## update_templates

Installs various templates from the `kicad_helpers/templates` directory (ignoring anything in the project's
`.gitignore` list). Similar to `nbdev`: [nbdev_new](https://nbdev.fast.ai/tutorial.html#Set-up-Repo). Templates
are stored in the [kicad_helpers/templates](https://github.com/ryanfobel/kicad-helpers/tree/main/kicad_helpers/templates)
folder, and are included with the python package by adding the followingline to the `MANIFEST.in` file:

```
graft kicad_helpers/templates
```

In [3]:
#export
@call_parse
def update_templates(v:Param("verbose", bool),
                     overwrite:Param("overwrite existing templates", bool),
                     root:Param("project root directory", str)="."):
    """
    Update project templates from the `kicad_helpers/templates` directory
    (ignoring anything in the project's `.gitignore` list).
    """
    templates_path = os.path.abspath(pkg_resources.resource_filename('kicad_helpers', 'templates'))
    root = _set_root(root)
    metadata = get_project_metadata(root)
    file_list = []
    exists_flag = False
    for root_, dirs, files in os.walk(templates_path):
        if len(files):
            for file in files:
                path = os.path.join(root_[len(templates_path) + 1:], file)
                if not in_gitignore(path):
                    src_path = os.path.abspath(os.path.join(templates_path, path))
                    dst_path = os.path.abspath(os.path.join(root, path))
                    
                    # Create the `dst_path` directory if it doesn't exist
                    os.makedirs(os.path.split(dst_path)[0], exist_ok=True)

                    if os.path.exists(dst_path):
                        if v:
                            print(f"{ path } already exists")
                            exists_flag = True
                        if not overwrite:
                            continue
    
                    with open(src_path) as f:
                        template = jinja2.Template(f.read())

                    with open(dst_path, "w") as f:
                        if v:
                            print(f"Rendering { path } template.")
                        f.write(template.render(**metadata))
                        
    if not overwrite and exists_flag:
        print("To overwrite existing files, use the --overwrite flag.")

In [4]:
# Install the patched version of kifield if it isn't already installed
# See: https://github.com/devbisme/KiField/issues/59#issuecomment-952298997
if subprocess.check_output([sys.executable, "-m", "kifield", "--help"]).decode("utf-8").find('addquantidty') > 0:
    install_python_package("git+https://github.com/ryanfobel/KiField.git@add-quantity")
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

```sh
kh_update --help
```

In [5]:
#hide_input
print(subprocess.check_output(f"kh_update --help", shell=True).decode("utf-8"))

usage: kh_update [-h] [--v] [--overwrite] [--root ROOT]

Update project templates from the `kicad_helpers/templates` directory (ignoring anything in the project's `.gitignore`
list).

optional arguments:
  -h, --help   show this help message and exit
  --v          verbose (default: False)
  --overwrite  overwrite existing templates (default: False)
  --root ROOT  project root directory (default: .)



```sh
kh_update --v --overwrite
```

In [6]:
#hide_input
print(subprocess.check_output(f"kh_update --v --overwrite --root { root }", shell=True).decode("utf-8"))

kitspace.yaml already exists
Rendering kitspace.yaml template.
settings.ini already exists
Rendering settings.ini template.
.github/workflows/build.yml already exists
Rendering .github/workflows/build.yml template.
.kicad_helpers_config/config.kibot.yaml already exists
Rendering .kicad_helpers_config/config.kibot.yaml template.
.kicad_helpers_config/manufacturers/PCBWay.kibot.yaml already exists
Rendering .kicad_helpers_config/manufacturers/PCBWay.kibot.yaml template.
tests/tests.ipynb already exists
Rendering tests/tests.ipynb template.



# Project structure

Here's an example showing the recommended KiCad project structure.

In [7]:
#hide input
print(subprocess.check_output(f"cd { root } && tree --dirsfirst -d -I $(python -c \"from kicad_helpers.utilities import get_gitignore_list; print('kicad_helpers|' + get_gitignore_list())\") .", shell=True).decode("utf-8"))

.
├── docs
│   ├── png
│   ├── src
│   └── svg
├── library
├── manufacturing
│   └── default
│       ├── assembly_photos
│       ├── gerbers
│       └── position
└── tests

11 directories



In [8]:
#export
@call_parse
def sch_to_bom(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """
    Update/create BOM from KiCad schematic.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield --nobackup --overwrite --group -aq -x { get_schematic_path(root) } -i { get_bom_path(root) }"
    if v:
        print(cmd)
    subprocess.check_output(cmd, shell=True)
    
@call_parse
def bom_to_sch(root:Param("project root directory", str)=".",
               v:Param("verbose", bool)=False,
               overwrite:Param("update existing schematic", bool)=False):
    """
    Update KiCad schematic from BOM file.
    """
    root = _set_root(root)
    cmd = f"{ sys.executable } -m kifield --nobackup --overwrite --fields ~quantity -x { get_bom_path(root) } -i { get_schematic_path(root) }"
    if v:
        print(cmd)
    subprocess.check_output(cmd, shell=True)

In [9]:
sch_to_bom(root=root)
df = pd.read_csv(get_bom_path(root))
df

,Refs,Quantity,MPN,Manufacturer,datasheet,footprint,value
0,"C1, C3, C5, C6, C8-C13, C16",11,CL21B104KBCNNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.1uF
1,"C14, C15",2,CL21C151JBANNNC,Samsung,NaN,Capacitors_SMD:C_0805,150pF
2,"C2, C4",2,T491D336K020AT,KEMET,NaN,Sci-Bots:SM2917,33uF
3,C7,1,CL21B103KCANNNC,Samsung,NaN,Capacitors_SMD:C_0805,0.01uF
4,DS1,1,150080BS75000,Würth Elektronik,NaN,LEDs:LED_0805,+3.3V_PWR
5,FB1-FB5,5,742792040,Würth Elektronik,NaN,Resistors_SMD:R_0805,FERRITE
6,JP1,1,NaN,NaN,NaN,Resistors_SMD:R_0805,JUMPER
7,P1,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Angled_1x06,CONN_01X06
8,P2,1,NaN,NaN,NaN,Pin_Headers:Pin_Header_Straight_2x03,CONN_01X06
9,P3-P6,4,NaN,NaN,NaN,Connect:1pin,CONN_01X01


In [10]:
bom_to_sch(root)